# Sample Generator
The aim of this python file is to generate samples of the payment arrivals

## Import Libraries

In [1]:
import concurrent.futures
import numpy as np
import pandas as pd
from scipy.stats import norm


## Random seed

In [3]:
rg = np.random.default_rng(12345)


## Parameters



In [2]:
# Prob of priority
_alpha = 0.5

# Number of banks
N = 15

# Number of time periods
T = 3_000

# Number of samples
S = 10_000

# Parameters of payment amounts
_mu = 1
_sigma = 0.25
K = 0.01

bank = []
time = []
payee = []
priority = []
n_sample = []
amount = []

Function to generate samples

In [4]:
def sample_generator(k, N, T, _alpha):
    
    _lambda = 1/N

    for i in range(N):
        t = 0
        # while loop for a day
        while t < T:
            ran_t = 0
            X_i = 0
            ran_payee = 0
            ran_pri = 0
            # generate order arrvial time
            ran_t = rg.random()
            t -= np.log(1 - ran_t)/ _lambda
            if t > T:
                break
            
            #generate random numbers to assign the variables
            X_i = rg.normal(_mu, _sigma, 1)
            if X_i[0] > 0:
                ran_amount = X_i[0]
            else:
                ran_amount = 0
                
            ran_pri = rg.random()
            if rg.random() >= _alpha:
                priority.append(0)
            else:
                priority.append(1)
                
            ran_payee = rg.random()
            p = int(ran_payee * (N - 1))
            if p >= i:
                p += 1
                
            #append the info to the list
            time.append(t)
            bank.append(i)
            payee.append(p)
            n_sample.append(k)
            amount.append(ran_amount)

Apply treading to save time

In [ ]:
with concurrent.futures.ThreadPoolExecutor() as executor:
    result = [executor.submit(sample_generator, k, N, T, _alpha) for k in range(S)]
    
    for future in concurrent.futures.as_completed(result):
        future.result()
        
samples = pd.DataFrame({'Sample': n_sample,	'Time': time, 'Bank': bank, 'Payee': payee, 
                        'Priority': priority, 'Amount': amount})

samples.sort_values(by=['Sample','Time'], ignore_index = True, inplace=True)
    

In [ ]:
# checking

samples['Amount'] = samples['Amount']*(1-K)/np.mean(samples['Amount']) + K


Export as csv file

In [ ]:
samples.to_csv(r'C:\Users\lucio\Desktop\paper\homobanks_0.5.csv', index=False) # please change %file path into your target file path